In [33]:
import requests
import pandas as pd
import json
from dateutil import parser

In [34]:
API_KEY = "eb37bea56c53b09e6f9d945964cea7c1-bfbf4e13bd8b3f2ccf15add1d4d8486e"
ACCOUNT_ID = "101-002-24455013-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [35]:
# create a new session variable where we can make our request
session = requests.Session()

In [36]:
# pass the headers through the session
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
})

In [37]:
# adding the parameters
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [38]:
# the url to get candles
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [39]:
# make the request and save it to response
response = session.get(url, params=None, data=None, headers=None)

In [40]:
# response status, ex, 200 for success
response.status_code

200

In [41]:
# convert response to json and save it to data
data = response.json()

In [42]:
data.keys()

dict_keys(['instruments', 'lastTransactionID'])

In [43]:
instruments_list = data['instruments']

In [44]:
# get length of all the instruments which are loaded
len(instruments_list)

123

In [45]:
# show only the first instrument from instruments_list
instruments_list[0]

{'name': 'NZD_CAD',
 'type': 'CURRENCY',
 'displayName': 'NZD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'minimumTradeSize': '1',
 'maximumTrailingStopDistance': '1.00000',
 'minimumTrailingStopDistance': '0.00050',
 'maximumPositionSize': '0',
 'maximumOrderUnits': '100000000',
 'marginRate': '0.03',
 'guaranteedStopLossOrderMode': 'DISABLED',
 'tags': [{'type': 'ASSET_CLASS', 'name': 'CURRENCY'}],
 'financing': {'longRate': '-0.0145',
  'shortRate': '-0.0125',
  'financingDaysOfWeek': [{'dayOfWeek': 'MONDAY', 'daysCharged': 1},
   {'dayOfWeek': 'TUESDAY', 'daysCharged': 1},
   {'dayOfWeek': 'WEDNESDAY', 'daysCharged': 1},
   {'dayOfWeek': 'THURSDAY', 'daysCharged': 1},
   {'dayOfWeek': 'FRIDAY', 'daysCharged': 1},
   {'dayOfWeek': 'SATURDAY', 'daysCharged': 0},
   {'dayOfWeek': 'SUNDAY', 'daysCharged': 0}]}}

In [46]:
# see all the keys in each instrument
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [47]:
# selecting only keys we are insterested in
key_i = ['name', 'type', 'displayName', 'pipLocation', 
         'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [48]:
# loop through all the instruments in the instruments_list
# for each loop, print out the "name" key value

# create an empty object first so we can fill it up when looping
instruments_dict = {}

# do the loop
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in key_i }

In [49]:
instruments_dict['NZD_CAD']

{'name': 'NZD_CAD',
 'type': 'CURRENCY',
 'displayName': 'NZD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.03'}

In [50]:
# save instruments_dict data in a new file
# new file: instruments.json
# file location in "data" folder, which is up ne level, so, ../
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [51]:
# create a function that will fetch candles
# example pair_name=NZD_CAD, count and granularity has default value in case none is provided
def fetch_candles(pair_name, count=10, granularity="H1"):
    # construct the url with the variables mentioned in above cells
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    # candles data may not be present, so...
    # when successful response, that is, 200:
    if response.status_code == 200:
        # return empty data array ELSE show candles data
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    # return a tuple: status_code & data. show me status_code and the contents in data
    return response.status_code, data

In [52]:
def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    # extract 'volume' and 'time' data and append it to final_data array
    # each 
    final_data = []
    # each instance inside 'data' is a 'candle'
    # each 'candle' therefore has the keys: complete, volume, time, bid, mid, ask
    for candle in data:
        # we dont take in last candle, that is, complete = FALSE
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    # when response code is not successful
    if code != 200:
        print("Failed", pair_name, data)
        return
    # when no candle data is loaded
    if len(data) == 0:
        print("No candles", pair_name)
    # when response is successfull AND there is candle data    
    candles_df = get_candles_df(data)
    # turn into a pickle file and save it to data folder
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    # show us what happened
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

In [53]:
create_data_file("NZD_CAD", count=20, granularity="H4")

NZD_CAD H4 20 candles, 2023-01-10 14:00:00+00:00 2023-01-13 18:00:00+00:00


In [54]:
# run an instance
code, data = fetch_candles("NZD_CAD", count=20)
candles_df = get_candles_df(data)


In [55]:
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2023-01-13 02:00:00+00:00,5349,0.85472,0.85481,0.85275,0.85402,0.85463,0.85471,0.85265,0.85392,0.85481,0.85492,0.85285,0.85412
1,2023-01-13 03:00:00+00:00,2917,0.85404,0.85406,0.85312,0.85355,0.85394,0.85396,0.85303,0.85346,0.85414,0.85416,0.85322,0.85364
2,2023-01-13 04:00:00+00:00,2385,0.85351,0.85370,0.85271,0.85318,0.85342,0.85361,0.85261,0.85307,0.85360,0.85380,0.85281,0.85328
3,2023-01-13 05:00:00+00:00,3159,0.85318,0.85376,0.85162,0.85202,0.85305,0.85365,0.85152,0.85192,0.85330,0.85388,0.85171,0.85213
4,2023-01-13 06:00:00+00:00,3344,0.85204,0.85422,0.85200,0.85389,0.85194,0.85412,0.85190,0.85380,0.85214,0.85432,0.85209,0.85398
5,2023-01-13 07:00:00+00:00,6169,0.85386,0.85387,0.85202,0.85219,0.85377,0.85378,0.85191,0.85209,0.85395,0.85396,0.85213,0.85229
6,2023-01-13 08:00:00+00:00,7567,0.85222,0.85458,0.85154,0.85396,0.85212,0.85448,0.85143,0.85387,0.85232,0.85467,0.85166,0.85405
7,2023-01-13 09:00:00+00:00,5815,0.85396,0.85510,0.85282,0.85347,0.85387,0.85500,0.85272,0.85337,0.85406,0.85519,0.85291,0.85357
8,2023-01-13 10:00:00+00:00,4749,0.85344,0.85434,0.85284,0.85290,0.85334,0.85424,0.85274,0.85281,0.85355,0.85443,0.85294,0.85299
9,2023-01-13 11:00:00+00:00,5370,0.85289,0.85297,0.85124,0.85146,0.85279,0.85288,0.85114,0.85135,0.85299,0.85306,0.85133,0.85156


In [56]:
candles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    20 non-null     datetime64[ns, tzutc()]
 1   volume  20 non-null     int64                  
 2   mid_o   20 non-null     float64                
 3   mid_h   20 non-null     float64                
 4   mid_l   20 non-null     float64                
 5   mid_c   20 non-null     float64                
 6   bid_o   20 non-null     float64                
 7   bid_h   20 non-null     float64                
 8   bid_l   20 non-null     float64                
 9   bid_c   20 non-null     float64                
 10  ask_o   20 non-null     float64                
 11  ask_h   20 non-null     float64                
 12  ask_l   20 non-null     float64                
 13  ask_c   20 non-null     float64                
dtypes: datetime64[ns, tzutc()](1), float64(12), 

In [57]:
our_curr = ['EUR', 'USD', 'GBP', 'CAD', 'AUD', 'NZD', 'JPY']

In [58]:
# list of all currency combinations coming from OANDA API
instruments_dict.keys()

dict_keys(['NZD_CAD', 'EUR_SGD', 'EU50_EUR', 'XAU_CAD', 'EUR_AUD', 'AU200_AUD', 'TRY_JPY', 'USD_SGD', 'EUR_SEK', 'AUD_CHF', 'SOYBN_USD', 'HKD_JPY', 'XAU_SGD', 'CORN_USD', 'GBP_AUD', 'USD_PLN', 'XAU_XAG', 'NAS100_USD', 'CAD_HKD', 'CH20_CHF', 'XAG_CHF', 'USD_CHF', 'XAG_HKD', 'AUD_HKD', 'NZD_CHF', 'ESPIX_EUR', 'GBP_CHF', 'USD_THB', 'XAU_JPY', 'XAU_HKD', 'GBP_CAD', 'EUR_HKD', 'CHF_JPY', 'GBP_HKD', 'EUR_NZD', 'XAG_AUD', 'WTICO_USD', 'XAG_NZD', 'AUD_SGD', 'TWIX_USD', 'EUR_JPY', 'EUR_TRY', 'USD_JPY', 'SGD_JPY', 'GBP_ZAR', 'XAG_JPY', 'ZAR_JPY', 'EUR_HUF', 'NZD_JPY', 'CN50_USD', 'CHF_ZAR', 'AUD_JPY', 'EUR_CHF', 'EUR_ZAR', 'CHINAH_HKD', 'NL25_EUR', 'USD_HKD', 'DE30_EUR', 'NZD_HKD', 'CAD_JPY', 'XAU_EUR', 'XPT_USD', 'SG30_SGD', 'EUR_USD', 'XAG_CAD', 'JP225_USD', 'EUR_CAD', 'USD_HUF', 'UK10YB_GBP', 'USD_MXN', 'GBP_USD', 'XAU_CHF', 'USD_DKK', 'XAU_NZD', 'USD_ZAR', 'XAU_GBP', 'USD_CZK', 'XAG_SGD', 'CAD_CHF', 'BCO_USD', 'EUR_DKK', 'USD_SEK', 'GBP_SGD', 'EUR_CZK', 'WHEAT_USD', 'XAU_AUD', 'CAD_SGD', 'AU

In [59]:
# create all forms of pairs possible with values in our_curr
# during each loop, feed the value to p1
for p1 in our_curr:
    # print(p1)
    # during each loop, feed the value to p2
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        # print(pr)
        # show instruments if "pr" matches any of the pairs from OANDA
        if pr in instruments_dict:
            # if matches, give me H1 and H4 candles
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4001 candles, 2022-05-25 00:00:00+00:00 2023-01-13 21:00:00+00:00
EUR_USD H4 4001 candles, 2020-06-22 01:00:00+00:00 2023-01-13 18:00:00+00:00
EUR_GBP H1 4001 candles, 2022-05-25 01:00:00+00:00 2023-01-13 21:00:00+00:00
EUR_GBP H4 4001 candles, 2020-06-22 01:00:00+00:00 2023-01-13 18:00:00+00:00
EUR_CAD H1 4001 candles, 2022-05-25 01:00:00+00:00 2023-01-13 21:00:00+00:00
EUR_CAD H4 4001 candles, 2020-06-22 01:00:00+00:00 2023-01-13 18:00:00+00:00
EUR_AUD H1 4001 candles, 2022-05-25 01:00:00+00:00 2023-01-13 21:00:00+00:00
EUR_AUD H4 4001 candles, 2020-06-22 01:00:00+00:00 2023-01-13 18:00:00+00:00
EUR_NZD H1 4001 candles, 2022-05-25 00:00:00+00:00 2023-01-13 21:00:00+00:00
EUR_NZD H4 4001 candles, 2020-06-22 01:00:00+00:00 2023-01-13 18:00:00+00:00
EUR_JPY H1 4001 candles, 2022-05-25 01:00:00+00:00 2023-01-13 21:00:00+00:00
EUR_JPY H4 4001 candles, 2020-06-19 17:00:00+00:00 2023-01-13 18:00:00+00:00
USD_CAD H1 4001 candles, 2022-05-25 01:00:00+00:00 2023-01-13 21:00:00+00:00